In [5]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load cleaned dataset
df = pd.read_csv("survey1_cleaned.csv")

# Separate features & target variable
X = df.drop(columns=['treatment'])  # Drop target column
y = df['treatment']  # Target variable

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (important for models like Logistic Regression)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Train & Evaluate models
best_model = None
best_f1 = 0
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Compute evaluation metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append((name, acc, prec, rec, f1))
    print(f"📊 {name} -> Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1-score: {f1:.3f}")

    # Save the best model
    if f1 > best_f1:
        best_f1 = f1
        best_model = model

# Save the best model to a .pkl file
with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

# Save the scaler to a .pkl file
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print(f"\n✅ Best Model: {best_model.__class__.__name__} (F1-score: {best_f1:.3f}) saved as 'best_model.pkl'")
print(f"✅ Scaler saved as 'scaler.pkl'")

📊 Logistic Regression -> Accuracy: 0.782, Precision: 0.747, Recall: 0.841, F1-score: 0.791
📊 Random Forest -> Accuracy: 0.862, Precision: 0.829, Recall: 0.911, F1-score: 0.902
📊 XGBoost -> Accuracy: 0.826, Precision: 0.790, Recall: 0.849, F1-score: 0.818

✅ Best Model: RandomForestClassifier (F1-score: 0.902) saved as 'best_model.pkl'
✅ Scaler saved as 'scaler.pkl'
